<a href="https://colab.research.google.com/github/PlagueDrGen/plague-dr-audio-suite/blob/main/PlagueD_Dr_Audio_Suite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: Language Packs (i18n)
# This dictionary holds all the text for the user interface.
# To add a new language, copy the "en" block, change the key (e.g., to "fr" for French),
# and translate all the text strings.

LANGUAGES = {
    "en": { # English
        # Tab Titles
        "tab_setup": "1. Setup",
        "tab_pipeline": "2. Mastering Pipeline",
        "tab_results": "3. Analysis & Results",
        "tab_watermark": "AI Watermark Disguise",
        # Setup Tab
        "setup_header": "## Welcome!",
        "setup_subheader": "Before you begin, please connect to your Google Drive account. The prompt will appear in the code output below.",
        "drive_button": "Connect to Google Drive",
        "drive_status_label": "Status",
        "drive_success_msg": "✅ Google Drive connected successfully. Ready to process audio.",
        "drive_error_msg": "❌ Error connecting to Google Drive: ",
        "drive_not_connected_error": "Google Drive not connected. Please go to the '1. Setup' tab and connect first.",
        # Pipeline Tab
        "pipeline_header": "## Mastering Pipeline",
        "pipeline_subheader": "Upload your audio file and select a mastering preset below, then click 'Run'.",
        "upload_label": "Upload Audio (MP3 or WAV)",
        "preset_label": "Select a Mastering Preset",
        "run_button": "Run Mastering Pipeline",
        # Results Tab
        "results_header": "## Analysis & A/B Comparison",
        "results_subheader": "Listen to the difference and compare the technical data.",
        "original_audio_label": "Original (Raw)",
        "mastered_audio_label": "Mastered",
        "original_stats_header": "### Original Audio Stats",
        "mastered_stats_header": "### Mastered Audio Stats",
        "original_stats_label": "Original Stats",
        "mastered_stats_label": "Mastered Stats",
        "details_header": "### Processing Details",
        "stems_label": "Generated Stem Files",
        "transcription_label": "Whisper Transcription",
        # Watermark Tab
        "watermark_header": "## AI Watermark Disguise",
        "watermark_subheader": "_This feature is under development._",
    },
    "es": { # Spanish
        # Tab Titles
        "tab_setup": "1. Configuración",
        "tab_pipeline": "2. Proceso de Masterización",
        "tab_results": "3. Análisis y Resultados",
        "tab_watermark": "Ocultador de Marcas de Agua (IA)",
        # Setup Tab
        "setup_header": "## ¡Bienvenido!",
        "setup_subheader": "Antes de comenzar, por favor conecte su cuenta de Google Drive. La solicitud aparecerá en la salida de la celda de código.",
        "drive_button": "Conectar a Google Drive",
        "drive_status_label": "Estado",
        "drive_success_msg": "✅ Google Drive conectado exitosamente. Listo para procesar audio.",
        "drive_error_msg": "❌ Error al conectar con Google Drive: ",
        "drive_not_connected_error": "Google Drive no está conectado. Por favor, vaya a la pestaña '1. Configuración' y conéctese primero.",
        # Pipeline Tab
        "pipeline_header": "## Proceso de Masterización",
        "pipeline_subheader": "Suba su archivo de audio y seleccione un preset de masterización, luego haga clic en 'Ejecutar'.",
        "upload_label": "Subir Audio (MP3 o WAV)",
        "preset_label": "Seleccione un Preset de Masterización",
        "run_button": "Ejecutar Proceso de Masterización",
        # Results Tab
        "results_header": "## Análisis y Comparación A/B",
        "results_subheader": "Escuche la diferencia y compare los datos técnicos.",
        "original_audio_label": "Original (Crudo)",
        "mastered_audio_label": "Masterizado",
        "original_stats_header": "### Estadísticas del Audio Original",
        "mastered_stats_header": "### Estadísticas del Audio Masterizado",
        "original_stats_label": "Estadísticas Originales",
        "mastered_stats_label": "Estadísticas Masterizadas",
        "details_header": "### Detalles del Procesamiento",
        "stems_label": "Archivos de Stems Generados",
        "transcription_label": "Transcripción de Whisper",
        # Watermark Tab
        "watermark_header": "## Ocultador de Marcas de Agua (IA)",
        "watermark_subheader": "_Esta función está en desarrollo._",
    }
}

print("✅ Cell 1 Complete: Language packs defined successfully.")

In [ ]:
# Cell 2: Install Libraries
# Run this cell once per session. It may take several minutes.

print("⏳ Installing required libraries... This may take a while.")
!pip install gradio pydub librosa openai-whisper soundfile pyloudnorm
!pip install -U demucs
print("✅ Cell 2 Complete: Libraries installed successfully.")

In [ ]:
# Cell 3: Main Application
# Run this cell after Cell 1 and Cell 2 are complete.
# This will launch the Gradio interface.

# 1. IMPORT LIBRARIES
# ------------------------------------------------------------------------------
import gradio as gr
from pydub import AudioSegment
from pydub.effects import normalize, compress_dynamic_range
import whisper
import torch
import os
import subprocess
import soundfile as sf
from google.colab import drive
import numpy as np
import pyloudnorm as pyln
import librosa


# 2. DEFINE ALL HELPER AND PROCESSING FUNCTIONS
# ------------------------------------------------------------------------------
def connect_google_drive():
    try:
        print("Connecting to Google Drive...")
        drive.mount('/content/drive')
        os.makedirs("/content/drive/MyDrive/PlagueDr_Output", exist_ok=True)
        print("Google Drive connected successfully.")
        return LANGUAGES["en"]["drive_success_msg"]
    except Exception as e:
        print(f"Error connecting to Google Drive: {e}")
        return f'{LANGUAGES["en"]["drive_error_msg"]}{e}'

def find_stem_files(output_dir, original_filename_base):
    stems_path = os.path.join(output_dir, "htdemucs", original_filename_base)
    if not os.path.exists(stems_path):
        return [f"Stem separation failed. Could not find output directory: {stems_path}"], None
    files = [os.path.join(stems_path, f) for f in os.listdir(stems_path)]
    vocal_file = next((f for f in files if "vocals" in f), None)
    return files, vocal_file

def analyze_audio(file_path):
    try:
        data, rate = sf.read(file_path)
        if data.size == 0: raise ValueError("Audio file is empty.")
        meter = pyln.Meter(rate)
        loudness = meter.integrated_loudness(data)
        abs_peak = np.max(np.abs(data))
        peak_dbfs = 20 * np.log10(abs_peak) if abs_peak > 0 else -np.inf
        rms = np.sqrt(np.mean(data**2))
        crest_factor = 20 * np.log10(abs_peak / rms) if rms > 0 else 0
        y, sr = librosa.load(file_path, sr=rate)
        tempo_frames, _ = librosa.beat.beat_track(y=y, sr=sr)
        tempo = np.mean(tempo_frames) if hasattr(tempo_frames, 'mean') else tempo_frames
        return {"Loudness (LUFS)": f"{loudness:.2f}", "Peak Level (dBFS)": f"{peak_dbfs:.2f}", "Dynamic Range (DR)": f"{crest_factor:.2f}", "Tempo (BPM)": f"{tempo:.2f}"}
    except Exception as e:
        print(f"Error analyzing file {file_path}: {e}")
        return {key: "Error" for key in ["Loudness (LUFS)", "Peak Level (dBFS)","Dynamic Range (DR)", "Tempo (BPM)"]}

def update_language(language_choice):
    lang_code = "es" if language_choice == "Español" else "en"
    lang = LANGUAGES[lang_code]
    return (
        gr.update(label=lang["tab_setup"]), gr.update(label=lang["tab_pipeline"]), gr.update(label=lang["tab_results"]), gr.update(label=lang["tab_watermark"]),
        gr.update(value=lang["setup_header"]), gr.update(value=lang["setup_subheader"]), gr.update(value=lang["drive_button"]), gr.update(label=lang["drive_status_label"]),
        gr.update(value=lang["pipeline_header"]), gr.update(value=lang["pipeline_subheader"]), gr.update(label=lang["upload_label"]), gr.update(label=lang["preset_label"]), gr.update(value=lang["run_button"]),
        gr.update(value=lang["results_header"]), gr.update(value=lang["results_subheader"]), gr.update(label=lang["original_audio_label"]), gr.update(label=lang["mastered_audio_label"]),
        gr.update(value=lang["original_stats_header"]), gr.update(value=lang["mastered_stats_header"]), gr.update(label=lang["original_stats_label"]), gr.update(label=lang["mastered_stats_label"]),
        gr.update(value=lang["details_header"]), gr.update(label=lang["stems_label"]), gr.update(label=lang["transcription_label"]),
        gr.update(value=lang["watermark_header"]), gr.update(value=lang["watermark_subheader"]),
    )

def run_full_pipeline(audio_file, preset_choice, lang_choice):
    lang_code = "es" if lang_choice == "Español" else "en"
    gdrive_output_dir = "/content/drive/MyDrive/PlagueDr_Output"
    if not os.path.exists('/content/drive/MyDrive'):
        error_message = LANGUAGES[lang_code]["drive_not_connected_error"]
        return None, None, {}, {}, error_message, error_message, gr.update(selected=0)
    if audio_file is None: return None, None, {}, {}, "Original audio not found.", "", gr.update(selected=1)
    print("Pipeline started...")
    sample_rate, audio_data = audio_file
    input_filename = "temp_uploaded_audio.wav"
    sf.write(input_filename, audio_data, sample_rate)
    original_filename_base = os.path.splitext(os.path.basename(input_filename))[0]
    print("Starting stem separation...")
    try:
        command = [ "python3", "-m", "demucs.separate", "--out", gdrive_output_dir, input_filename ]
        subprocess.run(command, check=True, capture_output=True, text=True)
    except subprocess.CalledProcessError as e:
        error_message = f"Demucs failed:\n{e.stderr}"
        return None, None, {}, {}, error_message, error_message, gr.update(selected=2)
    print("Stem separation complete.")
    stem_files, vocal_file_path = find_stem_files(gdrive_output_dir, original_filename_base)
    transcription_result = "Transcription unavailable."
    if vocal_file_path:
        print("Starting transcription...")
        model = whisper.load_model("base")
        result = model.transcribe(vocal_file_path)
        transcription_result = result["text"]
        print("Transcription complete.")
    print(f"Applying '{preset_choice}' preset...")
    song_to_master = AudioSegment.from_file(input_filename)
    # Mastering logic...
    if preset_choice == "Aggressive & Loud": mastered_song = normalize(compress_dynamic_range(song_to_master,-20.0,4.0).overlay(song_to_master.low_pass_filter(250)-2).overlay(song_to_master.high_pass_filter(4000)-3),0.1)
    elif preset_choice == "Clean & Bright": mastered_song = normalize(compress_dynamic_range(song_to_master,-15.0,2.5).overlay(song_to_master.high_pass_filter(6000)-6),0.1)
    elif preset_choice == "Warm & Vintage": mastered_song = normalize(compress_dynamic_range(song_to_master,-18.0,3.0).overlay(song_to_master.low_pass_filter(400).high_pass_filter(150)-4).low_pass_filter(18000),0.2)
    else: mastered_song = song_to_master
    mastered_filename = os.path.join(gdrive_output_dir, f"{original_filename_base}_{preset_choice.replace(' ', '_')}.wav")
    mastered_song.export(mastered_filename, format="wav")
    print("Mastering complete.")
    print("Analyzing files...")
    original_stats = analyze_audio(input_filename)
    mastered_stats = analyze_audio(mastered_filename)
    print("Analysis complete.")
    stems_output = "Stems saved to 'PlagueDr_Output' on Google Drive:\n\n" + "\n".join(stem_files)
    return input_filename, mastered_filename, original_stats, mastered_stats, stems_output, transcription_result, gr.update(selected=2)

# 4. DEFINE AND LAUNCH THE GRADIO USER INTERFACE
# ------------------------------------------------------------------------------

with gr.Blocks(theme=gr.themes.Soft(primary_hue="orange"), title="Plague Dr Audio Suite") as app:
    lang_selector = gr.Dropdown(label="Select Language / Seleccione el Idioma", choices=["English", "Español"], value="English")
    gr.Markdown("# Plague Dr Audio Suite")
    with gr.Tabs() as tabs:
        with gr.TabItem(LANGUAGES['en']['tab_setup'], id=0) as tab_setup:
            setup_header = gr.Markdown(LANGUAGES['en']['setup_header'])
            setup_subheader = gr.Markdown(LANGUAGES['en']['setup_subheader'])
            drive_button = gr.Button(LANGUAGES['en']['drive_button'], variant="primary")
            drive_status = gr.Textbox(label=LANGUAGES['en']['drive_status_label'], interactive=False)
        with gr.TabItem(LANGUAGES['en']['tab_pipeline'], id=1) as tab_pipeline:
            pipeline_header = gr.Markdown(LANGUAGES['en']['pipeline_header'])
            pipeline_subheader = gr.Markdown(LANGUAGES['en']['pipeline_subheader'])
            with gr.Row():
                with gr.Column():
                    pipeline_input = gr.Audio(label=LANGUAGES['en']['upload_label'], type="numpy")
                    preset_selector = gr.Dropdown(label=LANGUAGES['en']['preset_label'], choices=["Aggressive & Loud", "Clean & Bright", "Warm & Vintage"], value="Aggressive & Loud")
            pipeline_button = gr.Button(LANGUAGES['en']['run_button'], variant="primary")
        with gr.TabItem(LANGUAGES['en']['tab_results'], id=2) as tab_results:
            results_header = gr.Markdown(LANGUAGES['en']['results_header'])
            results_subheader = gr.Markdown(LANGUAGES['en']['results_subheader'])
            with gr.Row():
                original_audio_player = gr.Audio(label=LANGUAGES['en']['original_audio_label'])
                mastered_audio_player = gr.Audio(label=LANGUAGES['en']['mastered_audio_label'])
            with gr.Row():
                with gr.Column():
                    original_stats_header = gr.Markdown(LANGUAGES['en']['original_stats_header'])
                    original_stats_box = gr.JSON(label=LANGUAGES['en']['original_stats_label'])
                with gr.Column():
                    mastered_stats_header = gr.Markdown(LANGUAGES['en']['mastered_stats_header'])
                    mastered_stats_box = gr.JSON(label=LANGUAGES['en']['mastered_stats_label'])
            with gr.Row():
                with gr.Column():
                     details_header = gr.Markdown(LANGUAGES['en']['details_header'])
                     stems_output_box = gr.Textbox(label=LANGUAGES['en']['stems_label'], lines=6, interactive=False)
                     transcription_output_box = gr.Textbox(label=LANGUAGES['en']['transcription_label'], lines=6, interactive=False)
        with gr.TabItem(LANGUAGES['en']['tab_watermark'], id=4) as tab_watermark:
            watermark_header = gr.Markdown(LANGUAGES['en']['watermark_header'])
            watermark_subheader = gr.Markdown(LANGUAGES['en']['watermark_subheader'])

    all_components = [
        tab_setup, tab_pipeline, tab_results, tab_watermark, setup_header, setup_subheader, drive_button, drive_status,
        pipeline_header, pipeline_subheader, pipeline_input, preset_selector, pipeline_button, results_header, results_subheader,
        original_audio_player, mastered_audio_player, original_stats_header, mastered_stats_header, original_stats_box,
        mastered_stats_box, details_header, stems_output_box, transcription_output_box, watermark_header, watermark_subheader
    ]
    lang_selector.change(fn=update_language, inputs=lang_selector, outputs=all_components)
    drive_button.click(fn=connect_google_drive, inputs=None, outputs=drive_status)
    pipeline_button.click(
        fn=run_full_pipeline,
        inputs=[pipeline_input, preset_selector, lang_selector],
        outputs=[original_audio_player, mastered_audio_player, original_stats_box, mastered_stats_box, stems_output_box, transcription_output_box, tabs]
    )

print("\n🚀 Launching Gradio Interface...")
app.launch(debug=True)


🚀 Launching Gradio Interface...
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://dd5ebb4814e76b24e0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
